In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset
from PIL import Image
import torch.nn.functional as F

In [2]:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.9
log_interval = 10
img_size = 32

#Normalizing the data
shvn_transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(3)], [0.5 for _ in range(3)])])

mnist_transform = transforms.Compose(
    [transforms.Resize(img_size),
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

In [3]:
#FIRST 50% - SHVN
shvn_pre_trainset = datasets.SVHN(root='data/', split='train', download=True, transform=shvn_transform)
shvn_trainset = torch.utils.data.Subset(shvn_pre_trainset, range(25000))

shvn_pre_testset = datasets.SVHN(root='data/', split='test', download=True, transform=shvn_transform)
shvn_testset = torch.utils.data.Subset(shvn_pre_testset, range(5000))

#SECOND 50% - GENERATED MNIST
mnist_dataset = ImageFolder(root="../input/labelled-mnist/Labelled_MNIST", transform=mnist_transform)

mnist_trainset, mnist_testset = torch.utils.data.random_split(mnist_dataset, [25000, 5000], generator=torch.Generator().manual_seed(42))

#COMBINING TOGETHER
trainset = ConcatDataset([shvn_trainset, mnist_trainset])
train_loader = DataLoader(trainset, batch_size=batch_size_train, shuffle=True)

testset = ConcatDataset([shvn_testset, mnist_testset])
test_loader = DataLoader(testset, batch_size=batch_size_test, shuffle = True)

  0%|          | 0/182040794 [00:00<?, ?it/s]

  0%|          | 0/64275384 [00:00<?, ?it/s]

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding =0))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding =0))
        nn.Dropout(p=0.2),
        self.fc1 = nn.Linear(in_features=6*6*32, out_features=10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

cnn = CNN()

optimizer = optim.SGD(cnn.parameters(), lr=learning_rate, momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i * len(train_loader.dataset) for i in range (n_epochs + 1)]

In [5]:
def train(epoch): 
  cnn.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = cnn(data)

    loss_func = nn.CrossEntropyLoss()
    loss = loss_func(output, target)
    loss.backward()
    optimizer.step()

    #displaying the results
    if batch_idx % log_interval == 2:
      print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset)))

In [6]:
def test():
  cnn.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      output = cnn(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()

  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)

  print("\nTest set: Avg. loss: {:.4f}, Accuracy : {}/{} ({:.0f}%)\n".format(test_loss, correct, len(test_loader.dataset), 100.*correct/len(test_loader.dataset)))

In [7]:
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

Train Epoch: 1 [128/50000 (0%)]	Loss: 2.310387
Train Epoch: 1 [768/50000 (2%)]	Loss: 2.233346
Train Epoch: 1 [1408/50000 (3%)]	Loss: 1.949331
Train Epoch: 1 [2048/50000 (4%)]	Loss: 1.334427
Train Epoch: 1 [2688/50000 (5%)]	Loss: 1.449050
Train Epoch: 1 [3328/50000 (7%)]	Loss: 1.251245
Train Epoch: 1 [3968/50000 (8%)]	Loss: 1.257160
Train Epoch: 1 [4608/50000 (9%)]	Loss: 1.150416
Train Epoch: 1 [5248/50000 (10%)]	Loss: 1.225602
Train Epoch: 1 [5888/50000 (12%)]	Loss: 1.082041
Train Epoch: 1 [6528/50000 (13%)]	Loss: 0.981317
Train Epoch: 1 [7168/50000 (14%)]	Loss: 1.008680
Train Epoch: 1 [7808/50000 (16%)]	Loss: 1.200336
Train Epoch: 1 [8448/50000 (17%)]	Loss: 0.906843
Train Epoch: 1 [9088/50000 (18%)]	Loss: 0.982657
Train Epoch: 1 [9728/50000 (19%)]	Loss: 1.030773
Train Epoch: 1 [10368/50000 (21%)]	Loss: 0.867290
Train Epoch: 1 [11008/50000 (22%)]	Loss: 0.969375
Train Epoch: 1 [11648/50000 (23%)]	Loss: 0.702409
Train Epoch: 1 [12288/50000 (25%)]	Loss: 0.670876
Train Epoch: 1 [12928/5000

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: -9.3145, Accuracy : 8603/10000 (86%)

Train Epoch: 2 [128/50000 (0%)]	Loss: 0.313009
Train Epoch: 2 [768/50000 (2%)]	Loss: 0.487874
Train Epoch: 2 [1408/50000 (3%)]	Loss: 0.401497
Train Epoch: 2 [2048/50000 (4%)]	Loss: 0.290085
Train Epoch: 2 [2688/50000 (5%)]	Loss: 0.385866
Train Epoch: 2 [3328/50000 (7%)]	Loss: 0.270937
Train Epoch: 2 [3968/50000 (8%)]	Loss: 0.456370
Train Epoch: 2 [4608/50000 (9%)]	Loss: 0.384313
Train Epoch: 2 [5248/50000 (10%)]	Loss: 0.254800
Train Epoch: 2 [5888/50000 (12%)]	Loss: 0.390031
Train Epoch: 2 [6528/50000 (13%)]	Loss: 0.446445
Train Epoch: 2 [7168/50000 (14%)]	Loss: 0.385899
Train Epoch: 2 [7808/50000 (16%)]	Loss: 0.421201
Train Epoch: 2 [8448/50000 (17%)]	Loss: 0.197730
Train Epoch: 2 [9088/50000 (18%)]	Loss: 0.363362
Train Epoch: 2 [9728/50000 (19%)]	Loss: 0.567060
Train Epoch: 2 [10368/50000 (21%)]	Loss: 0.400047
Train Epoch: 2 [11008/50000 (22%)]	Loss: 0.346585
Train Epoch: 2 [11648/50000 (23%)]	Loss: 0.420229
Train Epoch: 2 [